# Extracting URLs from User Postings

In the following we load in the Postings datasets and extract the URLs and domains from them. Then we analyse the results and show how to merge the extracted data with the original dataset. First making the imports:

In [1]:
import pandas as pd
from urllib.parse import urlparse # for parsing the domains

<b>Loading datasets</b>

Importing the datasets from the input/ directory. These dataset do not exist in the GitLab repository currently. 

It turns out this directory is in .gitignore. I believe the files are meant to be copied from the repository to our local directories because the csv files are a bit large and it would take longer to clone the repository.

In [2]:
postings1 = pd.read_csv("../input/Postings_01052019_15052019.csv", delimiter=";")
postings2 = pd.read_csv("../input/Postings_16052019_31052019.csv", delimiter=";")

Merging the two columns together in order to make filtering easier, skipping the need for a loop and keeping the code more simple.

In [3]:
# merging headline and comment
comments1 = postings1["PostingHeadline"].fillna('') + " " + postings1["PostingComment"]
comments2 = postings2["PostingHeadline"].fillna('') + " " + postings2["PostingComment"]

<b>Extracting URLs and domains</b>

Copied the Regex from an online resource. This will not parse a url unless it starts with http(s)://. I believe this should be fine because I expect every proper URL link to include those.

In order to avoid making the handling of the dataset unnecessarily complicated, I'm assuming that there can be no more than one URL in a posting. 

In [4]:
url1 = comments1.str.extract(r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=]*))').dropna()[0]
url2 = comments2.str.extract(r'(https?:\/\/(?:www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b(?:[-a-zA-Z0-9()@:%_\+.~#?&\/=]*))').dropna()[0]

The urlparse() function from urllib.parse parses domains from urls. I've changed the output a little bit because urlparse would parse https://www.google.com/abc as www.google.com and https://google.com/abc as google.com, and thus those would be considered different domains. I believe that it makes sense to combine these as one, and thus I'm skipping everything before the final two '.'s.

In [5]:
domain1 = url1.apply(lambda x: ".".join(urlparse(x).netloc.split(".")[-2:]))
domain2 = url2.apply(lambda x: ".".join(urlparse(x).netloc.split(".")[-2:]))

I also attempted to retrieve the titles from the links but the following code takes forever to retrieve the titles.

If it makes sense to extract this information at some later point, I'll try and improve the code, but I'm leaving it out now.

In [6]:
## I tried to retrieve the titles for the URLs but it took forever. 
## I'll retry and improve this section at some later point if that makes sense.
# import requests
# def get_title(url):
#     try:
#         headers = {'headers':'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:51.0) Gecko/20100101 Firefox/51.0'}
#         n = requests.get(url, headers=headers)
#         al = n.text
#         return al[al.find('<title>') + 7 : al.find('</title>')]
#     except:
#         return None
# titles1 = url1.apply(lambda x: get_title(x))

Merging url and domain data into dataframes.

In [7]:
df1 = pd.DataFrame()
df1["url"] = url1
df1["domain"] = domain1

In [8]:
df2 = pd.DataFrame()
df2["url"] = url2
df2["domain"] = domain2

# Analyzing the extracted data

Let's first take a look at the sample of the data that was extracted to get a feel of the data that was extracted.

In [9]:
df1

,url,domain
0,https://www.youtube.com/watch?v=yiJ-sdjn2Zg,youtube.com
15,https://www.kleinezeitung.at/politik/innenpoli...,kleinezeitung.at
25,https://derstandard.at/1369363252227/Vom-hungr...,derstandard.at
32,https://wahlkabine.at/eu2019/wahlkabine/1,wahlkabine.at
44,https://derstandard.at/2000102257779/Vom-Umgan...,derstandard.at
...,...,...
343033,https://veganova.at/produkte/hag-capisco-mit-w...,veganova.at
343037,https://www.tagesschau.de/ausland/panamapapers...,tagesschau.de
343044,https://www.heise.de/tp/features/EU-Kommission...,heise.de
343046,https://derstandard.at/2000041068013/Wie-viel-...,derstandard.at


<b>Shares of postings with URLs</b>

As to be expected, not all the postings have a URL in them. The following code chunks show that roughly 3% of the posts have urls in them.

In [10]:
df1["url"].count()/comments1.count()

0.03732755599452003

In [11]:
df2["url"].count()/comments2.count()

0.03171388713862375

<b>Unique URLs and unique domains</b>

In the code chunks below we can see how only 10% of the posted URLs repeat, while it's true for 33% of the domains.

In [12]:
df1["url"].value_counts()[df1["url"].value_counts()==1].count()/df1["url"].value_counts().count()

0.8983776251617398

In [13]:
df1["domain"].value_counts()[df1["domain"].value_counts()==1].count()/df1["domain"].value_counts().count()

0.6748878923766816

# Saving the extracted data

Saving the extracted data so it can be loaded in a separate notebook or script without having to run the code in this file every time.

In [14]:
df1.to_csv("URLs-Postings_01052019_15052019.csv")

In [15]:
df2.to_csv("URLs-Postings_16052019_31052019.csv")

# Loading the data

In [16]:
# Code for loading in the data for the first Postings file
urls_df_1 = pd.read_csv("URLs-Postings_01052019_15052019.csv").set_index('Unnamed: 0')
urls_df_1.index.names = [None]

In [17]:
# Code for loading in the data for the second Postings file
urls_df_2 = pd.read_csv("URLs-Postings_16052019_31052019.csv").set_index('Unnamed: 0')
urls_df_2.index.names = [None]

<b>Viewing what we loaded in</b>

The following is identical to the df1 dataset we have seen before.

In [18]:
urls_df_1

,url,domain
0,https://www.youtube.com/watch?v=yiJ-sdjn2Zg,youtube.com
15,https://www.kleinezeitung.at/politik/innenpoli...,kleinezeitung.at
25,https://derstandard.at/1369363252227/Vom-hungr...,derstandard.at
32,https://wahlkabine.at/eu2019/wahlkabine/1,wahlkabine.at
44,https://derstandard.at/2000102257779/Vom-Umgan...,derstandard.at
...,...,...
343033,https://veganova.at/produkte/hag-capisco-mit-w...,veganova.at
343037,https://www.tagesschau.de/ausland/panamapapers...,tagesschau.de
343044,https://www.heise.de/tp/features/EU-Kommission...,heise.de
343046,https://derstandard.at/2000041068013/Wie-viel-...,derstandard.at


In [19]:
urls_df_1.equals(df1)

True

In [20]:
urls_df_2.equals(df2)

True

# Merging data with the original Postings datasets

For our task of using URLs as a possible way to measure similarity using the urls they posted, or whether any urls were posted in the first place, we need a user representing the user id. This ID is to be found in the Postings CSV files.

The following code merges the Postings CSV files with the urls_df_1 and urls_df_2 dataframes.

Remember we have previously loaded in "Postings_01052019_15052019.csv" as postings1 and "Postings_16052019_31052019.csv" as postings2 as dataframes.

In [21]:
merged1 = pd.concat([postings1, urls_df_1], axis=1)
merged2 = pd.concat([postings2, urls_df_2], axis=1)

Notice that the merged1 dataframe corresponds to the postings1 dataframe with additional columns "url" and "domain". 

The first row includes a URL and thus has some contents. Most other rows have NaN values there because they don't include any URLs.

In [22]:
merged1

,ID_Posting,ID_Posting_Parent,ID_CommunityIdentity,PostingHeadline,PostingComment,PostingCreatedAt,ID_Article,ArticlePublishingDate,ArticleTitle,ArticleChannel,ArticleRessortName,UserCommunityName,UserGender,UserCreatedAt,url,domain
0,1041073586,1.041073e+09,671476,Das hat gestern bereits der Voggenhuber angefü...,schieder hatte dem inhaltlich nichts entgegenz...,2019-05-01 18:21:15.127,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470,https://www.youtube.com/watch?v=yiJ-sdjn2Zg,youtube.com
1,1041073839,1.041073e+09,566938,NaN,...und meinen Bezirk bekommst du als Erbe mit.,2019-05-01 18:28:22.040,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,AlphaRomeo,m,2015-08-28 17:07:41.110,NaN,NaN
2,1041073872,1.041069e+09,669286,NaN,"Nein, bei der ÖVP/FPÖ genauso passiert. Ich wo...",2019-05-01 18:29:05.533,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Hpolditsch,NaN,2018-03-06 20:03:42.737,NaN,NaN
3,1041080734,1.041080e+09,671476,Sie haben doch nichts gefordert??,sie haben nur die regierung kritisiert. das di...,2019-05-01 22:37:56.010,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470,NaN,NaN
4,1041080828,NaN,671476,Heute wäre der perfekte Tag für die SPÖ gewese...,"ihr noch nicht erfülltes versprechen, den silb...",2019-05-01 22:42:06.310,2000102330973,2019-05-01 10:28:57.49,1. Mai in Wien: SPÖ fordert von Strache Rücktritt,Inland,Parteien,Ravenspower,NaN,2018-04-14 13:42:28.470,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
343155,1041171112,1.041171e+09,678438,NaN,Auch Ihnen besten Dank für Ihre Mühe! Ich bin...,2019-05-05 11:27:44.477,2000101751957,2019-05-05 10:00:00.00,"Österreichische Streamerinnen: ""Als Frau kämpf...",Web,Games,Ando,NaN,2018-08-15 17:06:45.757,NaN,NaN
343156,1041171278,NaN,678438,Eindeutig besser,Wie sich die elektronische Spielewelt doch ent...,2019-05-05 11:33:53.610,2000101751957,2019-05-05 10:00:00.00,"Österreichische Streamerinnen: ""Als Frau kämpf...",Web,Games,Ando,NaN,2018-08-15 17:06:45.757,NaN,NaN
343157,1041434423,1.041420e+09,116630,NaN,"aha, auch einer, ders nicht lassen kann.... wo...",2019-05-13 21:23:26.487,2000103031040,2019-05-13 12:00:46.94,Nach Großbrand in Wien-Simmering brauchen hund...,Panorama,Österreich,no me hables...,w,2006-04-25 15:49:57.257,NaN,NaN
343158,1041435327,NaN,687794,NaN,Hoffentlich sind keine Flüchtlinge unter den t...,2019-05-13 21:57:49.427,2000103031040,2019-05-13 12:00:46.94,Nach Großbrand in Wien-Simmering brauchen hund...,Panorama,Österreich,Afora,NaN,2019-02-13 21:52:02.820,NaN,NaN
